<a href="https://colab.research.google.com/github/shashi3876/machineLearning/blob/main/Updated_NHL_API_sample.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Get all available season IDs

In [1]:
import requests

res=requests.get('https://api-web.nhle.com/v1/standings-season')

seasonIds=[]
for seasonId in res.json()['seasons']:
  seasonIds.append(seasonId['id'])
print(seasonIds)
print(len(seasonIds))

[19171918, 19181919, 19191920, 19201921, 19211922, 19221923, 19231924, 19241925, 19251926, 19261927, 19271928, 19281929, 19291930, 19301931, 19311932, 19321933, 19331934, 19341935, 19351936, 19361937, 19371938, 19381939, 19391940, 19401941, 19411942, 19421943, 19431944, 19441945, 19451946, 19461947, 19471948, 19481949, 19491950, 19501951, 19511952, 19521953, 19531954, 19541955, 19551956, 19561957, 19571958, 19581959, 19591960, 19601961, 19611962, 19621963, 19631964, 19641965, 19651966, 19661967, 19671968, 19681969, 19691970, 19701971, 19711972, 19721973, 19731974, 19741975, 19751976, 19761977, 19771978, 19781979, 19791980, 19801981, 19811982, 19821983, 19831984, 19841985, 19851986, 19861987, 19871988, 19881989, 19891990, 19901991, 19911992, 19921993, 19931994, 19941995, 19951996, 19961997, 19971998, 19981999, 19992000, 20002001, 20012002, 20022003, 20032004, 20052006, 20062007, 20072008, 20082009, 20092010, 20102011, 20112012, 20122013, 20132014, 20142015, 20152016, 20162017, 20172018,

# Get all team codes

In [5]:
res=requests.get('https://api.nhle.com/stats/rest/en/team')
teamId=[]
teamDict={}
for team in res.json()['data']:
  teamId.append((team['rawTricode'],team['franchiseId'],team['fullName']))
  teamDict[team['rawTricode']]=team['fullName']
print(res.json())
print(teamId)
print(teamDict)
print(len(teamId))

{'data': [{'id': 11, 'franchiseId': 35, 'fullName': 'Atlanta Thrashers', 'leagueId': 133, 'rawTricode': 'ATL', 'triCode': 'ATL'}, {'id': 34, 'franchiseId': 26, 'fullName': 'Hartford Whalers', 'leagueId': 133, 'rawTricode': 'HFD', 'triCode': 'HFD'}, {'id': 31, 'franchiseId': 15, 'fullName': 'Minnesota North Stars', 'leagueId': 133, 'rawTricode': 'MNS', 'triCode': 'MNS'}, {'id': 32, 'franchiseId': 27, 'fullName': 'Quebec Nordiques', 'leagueId': 133, 'rawTricode': 'QUE', 'triCode': 'QUE'}, {'id': 33, 'franchiseId': 28, 'fullName': 'Winnipeg Jets (1979)', 'leagueId': 133, 'rawTricode': 'WIN', 'triCode': 'WIN'}, {'id': 35, 'franchiseId': 23, 'fullName': 'Colorado Rockies', 'leagueId': 133, 'rawTricode': 'CLR', 'triCode': 'CLR'}, {'id': 36, 'franchiseId': 3, 'fullName': 'Ottawa Senators (1917)', 'leagueId': 133, 'rawTricode': 'SEN', 'triCode': 'SEN'}, {'id': 37, 'franchiseId': 4, 'fullName': 'Hamilton Tigers', 'leagueId': 133, 'rawTricode': 'HAM', 'triCode': 'HAM'}, {'id': 38, 'franchiseId':

# Get Roaster for each year

In [ ]:
res=requests.get('https://api.nhle.com/stats/rest/en/team')
teamId=[]
for seasonId in res.json()['data']:
  teamId.append(seasonId['rawTricode'])
print(res.json())
print(teamId)
print(len(teamId))

In [ ]:
print(response.json)



<bound method Response.json of <Response [200]>>


In [ ]:
response.content

b'{"suggestions":[]}'

In [13]:
import requests
import json
import pandas as pd

# Lets you put in an NHL team id and season number
team_id = 16
season = "20192020"

# I think this loads a variable with the API site's text, then converts it to a dict that python can read
r = requests.get(base_url + str(team_id) + "?expand=team.roster&season=" + season)
r = requests.get(base_url + str(team_id) + "?expand=team.roster&season=*")
print(r)
data = json.loads(r.text)

<Response [404]>


JSONDecodeError: ignored

In [16]:
import requests

team_codes = []
# Get all teams
res = requests.get('https://api-web.nhle.com/v1/standings/now')
for team in res.json()['standings']:
    team_codes.append(team['teamAbbrev']['default'])

# Get all players
player_file = open('players.txt', "a")
url = 'https://api-web.nhle.com/v1/roster/{}/current'
for team in team_codes:
    res = requests.get(url.format(team))
    players = [player for position in ['forwards', 'defensemen', 'goalies'] for player in res.json()[position]]
    for player in players:
        player_file.write(f'{team} | {player["positionCode"]} {player["firstName"]["default"]} {player["lastName"]["default"]} {player["id"]}\n')
player_file.close()

In [5]:
import requests

team_codes = []
# Get all teams
res = requests.get('https://api-web.nhle.com/v1/standings/now')
for team in res.json()['standings']:
    team_codes.append(team['teamAbbrev']['default'])

print(res.json())
#
res=requests.get('https://api-web.nhle.com/v1/standings/season?20122013')
print(res.json())
for team in res.json()['seasons']:
  print(team)


{'wildCardIndicator': False, 'standings': [{'conferenceAbbrev': 'E', 'conferenceHomeSequence': 1, 'conferenceL10Sequence': 4, 'conferenceName': 'Eastern', 'conferenceRoadSequence': 1, 'conferenceSequence': 1, 'date': '2024-01-05', 'divisionAbbrev': 'M', 'divisionHomeSequence': 1, 'divisionL10Sequence': 3, 'divisionName': 'Metropolitan', 'divisionRoadSequence': 1, 'divisionSequence': 1, 'gameTypeId': 2, 'gamesPlayed': 37, 'goalDifferential': 24, 'goalDifferentialPctg': 0.648649, 'goalAgainst': 102, 'goalFor': 126, 'goalsForPctg': 3.405405, 'homeGamesPlayed': 18, 'homeGoalDifferential': 11, 'homeGoalsAgainst': 53, 'homeGoalsFor': 64, 'homeLosses': 5, 'homeOtLosses': 0, 'homePoints': 26, 'homeRegulationPlusOtWins': 12, 'homeRegulationWins': 11, 'homeTies': 0, 'homeWins': 13, 'l10GamesPlayed': 10, 'l10GoalDifferential': 13, 'l10GoalsAgainst': 24, 'l10GoalsFor': 37, 'l10Losses': 3, 'l10OtLosses': 0, 'l10Points': 14, 'l10RegulationPlusOtWins': 7, 'l10RegulationWins': 5, 'l10Ties': 0, 'l10Win

JSONDecodeError: ignored

In [14]:
url = 'https://api-web.nhle.com/v1/roster/BOS/20102011'
res=requests.get(url)
print(res.json())
#for team in res.json()['seasons']:
#  print(team)


{'forwards': [{'id': 8474657, 'headshot': 'https://assets.nhle.com/mugs/nhl/20102011/BOS/8474657.png', 'firstName': {'default': 'Jamie'}, 'lastName': {'default': 'Arniel'}, 'sweaterNumber': 72, 'positionCode': 'C', 'shootsCatches': 'R', 'heightInInches': 71, 'weightInPounds': 183, 'heightInCentimeters': 180, 'weightInKilograms': 83, 'birthDate': '1989-11-16', 'birthCity': {'default': 'Kingston'}, 'birthCountry': 'CAN', 'birthStateProvince': {'default': 'Ontario'}}, {'id': 8470638, 'headshot': 'https://assets.nhle.com/mugs/nhl/20102011/BOS/8470638.png', 'firstName': {'default': 'Patrice'}, 'lastName': {'default': 'Bergeron'}, 'sweaterNumber': 37, 'positionCode': 'C', 'shootsCatches': 'R', 'heightInInches': 73, 'weightInPounds': 196, 'heightInCentimeters': 185, 'weightInKilograms': 89, 'birthDate': '1985-07-24', 'birthCity': {'default': "L'Ancienne-Lorette"}, 'birthCountry': 'CAN', 'birthStateProvince': {'default': 'Quebec', 'fr': 'Québec'}}, {'id': 8470169, 'headshot': 'https://assets.n

In [1]:
len(team_codes)

NameError: ignored